In [ ]:
##The purpose of this file is to gather all raw data we scraped from all departments and topic into one csv file, remember that is still the raw data

In [1]:
import requests
import random
import time
import pandas as pd
import numpy as np
import re
import datetime
import nltk
from bs4 import BeautifulSoup
import string
import socket

In [2]:
pd.set_option('display.max_colwidth', 150)
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()
wn = nltk.WordNetLemmatizer()
socket.setdefaulttimeout(20)

In [3]:
user_agent_list = [
'Mozilla/5.0 (X11; Linux i686) AppleWebKit/534.30 (KHTML, like Gecko) Slackware/Chrome/12.0.742.100 Safari/534.30',
'Mozilla/5.0 (X11; Linux i686) AppleWebKit/534.30 (KHTML, like Gecko) Chrome/12.0.742.100 Safari/534.30',
'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/534.30 (KHTML, like Gecko) Chrome/12.0.742.100 Safari/534.30',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_0) AppleWebKit/534.30 (KHTML, like Gecko) Chrome/12.0.742.100 Safari/534.30',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_4) AppleWebKit/534.30 (KHTML, like Gecko) Chrome/12.0.742.100 Safari/534.30',
'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.30 (KHTML, like Gecko) Chrome/12.0.724.100 Safari/534.30',
'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/534.25 (KHTML, like Gecko) Chrome/12.0.706.0 Safari/534.25',
'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/534.25 (KHTML, like Gecko) Chrome/12.0.704.0 Safari/534.25',
'Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US) AppleWebKit/525.13 (KHTML, like Gecko) Chrome/0.2.149.27 Safari/525.13',
'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/525.13 (KHTML, like Gecko) Chrome/0.2.149.27 Safari/525.13',
'Mozilla/5.0 (Windows; U; Windows NT 5.0; en-US) AppleWebKit/525.13 (KHTML, like Gecko) Chrome/0.2.149.27 Safari/525.13',
'Mozilla/5.0 (Linux; U; en-US) AppleWebKit/525.13 (KHTML, like Gecko) Chrome/0.2.149.27 Safari/525.13',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
'Mozilla/5.0 (Windows 8) AppleWebKit/534.30 (KHTML, like Gecko) Chrome/12.0.742.112 Safari/534.30',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_6) AppleWebKit/534.30 (KHTML, like Gecko) Chrome/12.0.742.112 Safari/534.30',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_4) AppleWebKit/534.30 (KHTML, like Gecko) Chrome/12.0.742.112 Safari/534.30',
'Mozilla/5.0 (X11; CrOS i686 12.433.216) AppleWebKit/534.30 (KHTML, like Gecko) Chrome/12.0.742.105 Safari/534.30',
'Mozilla/5.0 ArchLinux (X11; U; Linux x86_64; en-US) AppleWebKit/534.30 (KHTML, like Gecko) Chrome/12.0.742.100 Safari/534.30',
'Mozilla/5.0 ArchLinux (X11; U; Linux x86_64; en-US) AppleWebKit/534.30 (KHTML, like Gecko) Chrome/12.0.742.100',
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
'Mozilla/5.0 (X11; Linux i686) AppleWebKit/534.30 (KHTML, like Gecko) Ubuntu/10.04 Chromium/12.0.742.112 Chrome/12.0.742.112 Safari/534.30',
'Mozilla/5.0 (Windows NT 7.1) AppleWebKit/534.30 (KHTML, like Gecko) Chrome/12.0.742.112 Safari/534.30',
'Mozilla/5.0 (Windows NT 5.2) AppleWebKit/534.30 (KHTML, like Gecko) Chrome/12.0.742.112 Safari/534.30',
'Mozilla/5.0 (Macintosh; U; Mac OS X 10_6_1; en-US) AppleWebKit/530.5 (KHTML, like Gecko) Chrome/ Safari/530.5',
'Mozilla/5.0 (Macintosh; U; Mac OS X 10_5_7; en-US) AppleWebKit/530.5 (KHTML, like Gecko) Chrome/ Safari/530.5',
'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_5_6; en-US) AppleWebKit/530.9 (KHTML, like Gecko) Chrome/ Safari/530.9',
'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_5_6; en-US) AppleWebKit/530.6 (KHTML, like Gecko) Chrome/ Safari/530.6',
'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_5_6; en-US) AppleWebKit/530.5 (KHTML, like Gecko) Chrome/ Safari/530.5',
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36',
'Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2919.83 Safari/537.36',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2866.71 Safari/537.36',
'Mozilla/5.0 (X11; Ubuntu; Linux i686 on x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2820.59 Safari/537.36',
'Mozilla/5.0 (Windows; U; Windows NT 6.0; de) AppleWebKit/525.13 (KHTML, like Gecko) Chrome/0.2.149.27 Safari/525.13',
'Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/525.13 (KHTML, like Gecko) Chrome/0.2.149.27 Safari/525.13',
'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/525.13(KHTML, like Gecko) Chrome/0.2.149.27 Safari/525.13',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2762.73 Safari/537.36',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2656.18 Safari/537.36',
'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML like Gecko) Chrome/44.0.2403.155 Safari/537.36',
'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36',
]

In [4]:
df0 = pd.read_csv('8.10_All_Cancer_center_raw.csv')

In [7]:
df0

,Provider_name,Department,Researcher_Type,Profile_link,Provider_all_top_concepts,Top_Concepts
0,Steven Grossman,Cancer Center,Professor,https://researcherprofiles.org/profile/133498396,https://profiles.sc-ctsi.org/display/133498396/network/researchareas/timeline,"Fatigue, 1 publications between 2008 and 2008, average publication date April 2008. Pain, 1 publications between 2008 and 2008, average publicatio..."
1,"Christopher Haiman, ScD",Cancer Center,Professor,https://researcherprofiles.org/profile/177063,https://profiles.sc-ctsi.org/display/177063/network/researchareas/timeline,"Chromosomes, Human, Pair 8, 21 publications between 2006 and 2020, average publication date December 2011. Linkage Disequilibrium, 55 publications..."
2,Caryn Lerman,Cancer Center,Professor,https://researcherprofiles.org/profile/41128129,https://profiles.sc-ctsi.org/display/41128129/network/researchareas/timeline,"Genetic Testing, 46 publications between 1993 and 2020, average publication date July 2000. Genetic Counseling, 39 publications between 1991 and 2..."


In [ ]:
## in my case, the "All_Else_raw.csv" is a csv file I already initialized to append all data, then what I do is read and append all other csv files data into that one

In [8]:
df0.to_csv("All_Else_raw.csv", index=False, header=True)

In [9]:
df1 = pd.read_csv('8.10_All_Neurological_Surgery_raw.csv')

In [10]:
df1.to_csv("All_Else_raw.csv", mode='a',index=False, header=False)

In [11]:
df2 = pd.read_csv('8.10_All_Ophthalmology_raw.csv')

In [12]:
df2.to_csv("All_Else_raw.csv", mode='a',index=False, header=False)

In [13]:
df3 = pd.read_csv('8.10_All_Orthopaedic_Surgery_raw.csv')

In [14]:
df3.to_csv("All_Else_raw.csv", mode='a',index=False, header=False)

In [15]:
df4 = pd.read_csv('8.10_All_Otolaryngology_raw.csv')

In [16]:
df4.to_csv("All_Else_raw.csv", mode='a',index=False, header=False)

In [19]:
df5 = pd.read_csv('8.10_All_Radiation_Oncology_raw.csv')

In [20]:
df5.to_csv("All_Else_raw.csv", mode='a',index=False, header=False)

In [21]:
df6 = pd.read_csv('8.10_All_surgery_raw.csv')

In [22]:
df6.to_csv("All_Else_raw.csv", mode='a',index=False, header=False)

In [23]:
df7 = pd.read_csv('8.10_All_Urology_raw.csv')

In [24]:
df7.to_csv("All_Else_raw.csv", mode='a',index=False, header=False)

In [ ]:
#####################################################

In [25]:
dfraw = pd.read_csv('All_Else_raw.csv')

In [26]:
dfraw

,Provider_name,Department,Researcher_Type,Profile_link,Provider_all_top_concepts,Top_Concepts
0,Steven Grossman,Cancer Center,Professor,https://researcherprofiles.org/profile/133498396,https://profiles.sc-ctsi.org/display/133498396/network/researchareas/timeline,"Fatigue, 1 publications between 2008 and 2008, average publication date April 2008. Pain, 1 publications between 2008 and 2008, average publicatio..."
1,"Christopher Haiman, ScD",Cancer Center,Professor,https://researcherprofiles.org/profile/177063,https://profiles.sc-ctsi.org/display/177063/network/researchareas/timeline,"Chromosomes, Human, Pair 8, 21 publications between 2006 and 2020, average publication date December 2011. Linkage Disequilibrium, 55 publications..."
2,Caryn Lerman,Cancer Center,Professor,https://researcherprofiles.org/profile/41128129,https://profiles.sc-ctsi.org/display/41128129/network/researchareas/timeline,"Genetic Testing, 46 publications between 1993 and 2020, average publication date July 2000. Genetic Counseling, 39 publications between 1991 and 2..."
3,"Arun P. Amar, MD",Neurological Surgery,Associate Professor,https://researcherprofiles.org/profile/197343,https://profiles.sc-ctsi.org/display/197343/network/researchareas/timeline,"Electric Stimulation Therapy, 6 publications between 1998 and 2007, average publication date March 2002. Vagus Nerve, 8 publications between 1998 ..."
4,"Patrick Hsieh, MD",Neurological Surgery,Professor,https://researcherprofiles.org/profile/178188,https://profiles.sc-ctsi.org/display/178188/network/researchareas/timeline,"Osteotomy, 6 publications between 2007 and 2014, average publication date August 2010. Sacrum, 4 publications between 2007 and 2014, average publi..."
...,...,...,...,...,...,...
318,"Shilo Rosenberg, MD",Urology,Assistant Professor,https://researcherprofiles.org/profile/177614,https://profiles.sc-ctsi.org/display/177614/network/researchareas/timeline,"Seminal Vesicles, 1 publications between 2009 and 2009, average publication date January 2009. Multiple Myeloma, 1 publications between 2010 and 2..."
319,"Mary Samplaski, MD",Urology,Associate Professor,https://researcherprofiles.org/profile/191141,https://profiles.sc-ctsi.org/display/191141/network/researchareas/timeline,"Prostatitis, 2 publications between 2011 and 2012, average publication date January 2012. Sperm Motility, 3 publications between 2010 and 2021, av..."
320,"Anne Kathryn Schuckman, MD",Urology,Associate Professor,https://researcherprofiles.org/profile/197093,https://profiles.sc-ctsi.org/display/197093/network/researchareas/timeline,"Urologic Neoplasms, 2 publications between 2009 and 2020, average publication date January 2015. Carcinoma, Squamous Cell, 4 publications between ..."
321,"Rene Sotelo, MD",Urology,Professor,https://researcherprofiles.org/profile/197422,https://profiles.sc-ctsi.org/display/197422/network/researchareas/timeline,"Aged, 1 publications between 2019 and 2019, average publication date June 2019. Cystoscopy, 1 publications between 2019 and 2019, average publicat..."
